In [1]:
import sys
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import os
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
import argparse
from accelerometer import utils
import matplotlib

/home/aayush/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# http://pandas-docs.github.io/pandas-docs-travis/whatsnew/v0.21.1.html#restore-matplotlib-datetime-converter-registration
register_matplotlib_converters()

LABELS_AND_COLORS = {
    'imputed': '#fafc6f',
    'sleep': 'midnightblue',
    'sit-stand': 'red',
    'sedentary': 'red',
    'vehicle': 'saddlebrown',
    'light': 'darkorange',
    'mixed': 'seagreen',
    'walking': 'limegreen',
    'moderate-vigorous': 'green',
    'moderate': 'green',
    'vigorous': 'springgreen',
    'bicycling': 'springgreen',
    'tasks-light': 'darkorange',
    'SB': 'red',  # sedentary behaviour
    'LPA': 'darkorange',  # light physical activity
    'MVPA': 'green',  # moderate-vigorous physical activity
    'MPA': 'green',  # moderate physical activity
    'VPA': 'springgreen',  # vigorous physical activity
}


In [24]:
data = pd.read_csv(
    "/home/aayush/accelerometer/accprocess/results/P037-timeSeries.csv.gz",
    index_col='time',
    parse_dates=['time'],
    date_parser=utils.date_parser
)

In [25]:
data.iloc[100]

acc                  7.744
light                0.000
moderate-vigorous    0.000
sedentary            0.000
sleep                1.000
CpSB                 0.000
CpLPA                0.000
CpMPA                0.000
CpVPA                0.000
CpMVPA               0.000
MET                  0.950
Name: 2016-10-30 01:18:00+01:00, dtype: float64

In [26]:
# use tz-naive local time

# this part is taking care of daylight savings time and moving time forward by an hour leading to a gap in the plot.
if data.index.tz is not None:
    data.index = (
        # convoluted way to mask the ambiguous DST pushback hour, if any
        data.index
        .tz_localize(None)
        .tz_localize(data.index.tz, ambiguous='NaT', nonexistent='NaT')
        .tz_localize(None)
    )
    data = data[data.index.notnull()]

In [33]:
data.iloc[62]

acc                  5.912
light                0.000
moderate-vigorous    0.000
sedentary            0.000
sleep                1.000
CpSB                 0.000
CpLPA                0.000
CpMPA                0.000
CpVPA                0.000
CpMVPA               0.000
MET                  0.950
Name: 2016-10-30 00:59:00, dtype: float64

In [15]:
# fix gaps or irregular sampling
if pd.infer_freq(data) is None:
    freq = pd.infer_freq(data.head()) or '30s'  # try to infer from first few rows, else default to 30s
    new_index = pd.date_range(data.index[0], data.index[-1], freq=freq)
    data = data.reindex(new_index, method='nearest', tolerance=freq, fill_value=np.NaN)

In [16]:
data

,acc,light,moderate-vigorous,sedentary,sleep,CpSB,CpLPA,CpMPA,CpVPA,CpMVPA,MET
2016-10-30 00:28:00,1.805,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.95
2016-10-30 00:28:30,1.759,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.95
2016-10-30 00:29:00,1.702,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.95
2016-10-30 00:29:30,1.815,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.95
2016-10-30 00:30:00,1.672,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.95
...,...,...,...,...,...,...,...,...,...,...,...
2016-10-31 02:55:30,1.455,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.95
2016-10-31 02:56:00,1.430,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.95
2016-10-31 02:56:30,1.390,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.95
2016-10-31 02:57:00,1.492,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.95
